In [1]:
pip install transformers torch imaplib2 email-validator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import imaplib
import email
from email.header import decode_header
import smtplib
from email.mime.text import MIMEText
from transformers import pipeline

# 1. Connect to email and read the latest UNSEEN message
def fetch_latest_email(user, password):
    imap_server = 'imap.gmail.com'
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(user, password)
    mail.select("inbox")

    status, messages = mail.search(None, 'UNSEEN')  # 'ALL' for all mails
    email_ids = messages[0].split()

    if not email_ids:
        return None, None

    latest_email_id = email_ids[-1]
    status, msg_data = mail.fetch(latest_email_id, "(RFC822)")
    raw_email = msg_data[0][1]
    msg = email.message_from_bytes(raw_email)

    sender = msg["From"]
    subject = decode_header(msg["Subject"])[0][0]
    if isinstance(subject, bytes):
        subject = subject.decode()

    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True).decode()
                break
            elif part.get_content_type() == "text/html":
              # Optional: Handle HTML part as well if needed
              # body = part.get_payload(decode=True).decode()
              pass
    else:
        body = msg.get_payload(decode=True).decode()

    return body.strip(), sender

# 2. Load open-source model
reply_generator = pipeline("text2text-generation", model="google/flan-t5-base")

def generate_reply(email_text):
    prompt = f"Write a professional email reply to:\n\n{email_text}"
    response = reply_generator(prompt, max_length=200, do_sample=True)
    return response[0]["generated_text"]

# 3. Send the reply
def send_reply_email(reply_text, recipient, user_email, user_password):
    msg = MIMEText(reply_text)
    msg['Subject'] = "Re: Your Email"
    msg['From'] = user_email
    msg['To'] = recipient

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(user_email, user_password)
        server.sendmail(user_email, recipient, msg.as_string())

# 4. Main pipeline
def auto_email_responder(user_email, user_password):
    email_text, sender = fetch_latest_email(user_email, user_password)
    if email_text:
        print(f"Incoming Email:\n{email_text}\n\nGenerating reply...")
        reply = generate_reply(email_text)
        print(f"\nGenerated Reply:\n{reply}")
        send_reply_email(reply, sender, user_email, user_password)
        print("Reply sent successfully!")
    else:
        print("No new emails.")

# Replace with your credentials
user_email = "prabathara143@gmail.com"
user_password = "dqxk upup cxpb mmyz"  # Use App Password if Gmail with 2FA

auto_email_responder(user_email, user_password)

Device set to use cuda:0


Incoming Email:
[image: Google]
App password created to sign in to your account


prabathara143@gmail.com
If you didn't generate this password for Gmail, someone might be using your
account. Check and secure your account now.
Check activity
<https://accounts.google.com/AccountChooser?Email=prabathara143@gmail.com&continue=https://myaccount.google.com/alert/nt/1753760959129?rfn%3D20%26rfnc%3D1%26eid%3D-6216317082988162308%26et%3D0>
You can also see security activity at
https://myaccount.google.com/notifications
You received this email to let you know about important changes to your
Google Account and services.
© 2025 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA

Generating reply...


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Generated Reply:
prabathara143@gmail.com If you didn't generate this password for Gmail, someone might be using your account now. Check and secure your account now. Check activity https://accounts.google.com/AccountChooser?Email=prabathara143@gmail.com&continue=https://myaccount.google.com/alert/nt/1753760959129?rfn%3D20%26rfnc%3D1%26eid%3D-6216317082988162308%26et%3D0> You can also see security activity at https://myaccount.google.com/notifications
Reply sent successfully!


In [5]:
pip install gradio transformers torch


In [6]:
import gradio as gr
from transformers import pipeline

# Load model (open-source, instruction-tuned)
reply_generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Reply logic with prompt engineering
def generate_reply(email_text):
    prompt = (
        "You are an AI email assistant. Write a short, professional reply to the email below.\n\n"
        f"Email: {email_text}\n\n"
        "Reply:"
    )
    response = reply_generator(prompt, max_new_tokens=200, do_sample=True)
    return response[0]["generated_text"]

# Gradio UI
iface = gr.Interface(
    fn=generate_reply,
    inputs=gr.Textbox(label="Paste Incoming Email Here", lines=15, placeholder="Hi, could you send the report by Friday?"),
    outputs=gr.Textbox(label="AI Generated Reply"),
    title="📧 Email Reply Generator",
    description="Paste any email and get a professional AI-generated response using the open-source FLAN-T5 model."
)

iface.launch()


Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69e3724cbc899823c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
